In [2]:
import pandas as pd

data = pd.read_csv('../../csv/train.csv', index_col='PassengerId')
test = pd.read_csv('../../csv/test.csv')

In [3]:
data['is_male'] = (data['Sex'] == 'male').astype(int)
data['is_female'] = (data['Sex'] == 'female').astype(int)

test['is_male'] = (test['Sex'] == 'male').astype(int)
test['is_female'] = (test['Sex'] == 'female').astype(int)

data['Age'] = data['Age'].fillna(0)
test['Age'] = test['Age'].fillna(0)

In [4]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,is_male,is_female
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q,1,0
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S,0,1
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q,1,0
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S,1,0
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S,0,1


In [5]:
data = data.drop(columns=['Name', 'Ticket', 'Cabin', 'Embarked', 'Sex'])
test = test.drop(columns=['Name', 'Ticket', 'Cabin', 'Embarked', 'Sex'])

In [6]:
data.head()

,Survived,Pclass,Age,SibSp,Parch,Fare,is_male,is_female
PassengerId,,,,,,,,
1,0,3,22.0,1,0,7.2500,1,0
2,1,1,38.0,1,0,71.2833,0,1
3,1,3,26.0,0,0,7.9250,0,1
4,1,1,35.0,1,0,53.1000,0,1
5,0,3,35.0,0,0,8.0500,1,0


In [7]:
X = data.drop(columns=['Survived'])
y = data['Survived']

In [8]:
# random forest
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

for n_trees in [10, 25, 50, 100, 200, 500]:
    forest = RandomForestClassifier(random_state=3, n_estimators=n_trees, n_jobs=-1)
    forest.fit(X, y)
    y_predicted_train = forest.predict(X)

    print('-- ' * 10)
    print("Train:", accuracy_score(y, y_predicted_train))

-- -- -- -- -- -- -- -- -- -- 
Train: 0.9629629629629629
-- -- -- -- -- -- -- -- -- -- 
Train: 0.9820426487093153
-- -- -- -- -- -- -- -- -- -- 
Train: 0.9820426487093153
-- -- -- -- -- -- -- -- -- -- 
Train: 0.9820426487093153
-- -- -- -- -- -- -- -- -- -- 
Train: 0.9820426487093153
-- -- -- -- -- -- -- -- -- -- 
Train: 0.9820426487093153


In [9]:
test['Pclass'] = test['Pclass'].fillna(0)
test['SibSp'] = test['SibSp'].fillna(0)
test['Parch'] = test['Parch'].fillna(0)
test['Fare'] = test['Fare'].fillna(0)
test.head()

,PassengerId,Pclass,Age,SibSp,Parch,Fare,is_male,is_female
0,892,3,34.5,0,0,7.8292,1,0
1,893,3,47.0,1,0,7.0000,0,1
2,894,2,62.0,0,0,9.6875,1,0
3,895,3,27.0,0,0,8.6625,1,0
4,896,3,22.0,1,1,12.2875,0,1


In [10]:
from sklearn.ensemble import RandomForestClassifier

forest = RandomForestClassifier(random_state=3, n_estimators=10, n_jobs=-1, max_depth=10)
y_predicted_test = forest.predict_proba(test.drop(columns=['PassengerId']))[:, 1] >= 0.7
# predicted = tree.predict_proba(X_test)[:, 1] >= 0.3

y_predicted_test = [[0, 1][y_predicted_test[i]] for i in range(len(y_predicted_test))]
print(y_predicted_test)

submission = pd.DataFrame({'PassengerId':test['PassengerId'],'Survived':y_predicted_test})
submission.head(5)

NotFittedError: This RandomForestClassifier instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [ ]:
filename = 'forest-titanic-predictions.csv'

submission.to_csv(filename, index=False)

print('Saved file: ' + filename)